## 📘 Notebook Overview: 01_data_cleaning_preparation.ipynb

This notebook performs the initial steps of the DataSpark project including:

💾 **Loading Data**  
📁 Loading datasets from the `data/raw/` directory.

🧼 **Data Cleaning**  
- Handling missing values  
- Normalizing column names  
- Fixing incorrect data types  
- Parsing date fields and currency values

🔗 **Data Merging**  
Combining customer, sales, product, store, and currency data.

💡 **Tip:** Keep cleaned data in `data/clean/` for reproducibility and use in SQL/Power BI.  
🚨 **Warning:** Always check for null values before merging tables.  
✅ **Output:** Cleaned and saved datasets, ready for analysis and visualization.


In [ ]:
import pandas as pd
import os

# Set path to raw data folder
raw_data_path = "../data/raw"

# Load all datasets
customers = pd.read_csv(os.path.join(raw_data_path, "Customers.csv"), encoding='ISO-8859-1')
products = pd.read_csv(os.path.join(raw_data_path, "Products.csv"), encoding='ISO-8859-1')
sales = pd.read_csv(os.path.join(raw_data_path, "Sales.csv"), encoding='ISO-8859-1')
stores = pd.read_csv(os.path.join(raw_data_path, "Stores.csv"), encoding='ISO-8859-1')
exchange_rates = pd.read_csv(os.path.join(raw_data_path, "Exchange_Rates.csv"), encoding='ISO-8859-1')
data_dict = pd.read_csv(os.path.join(raw_data_path, "Data_Dictionary.csv"), encoding='ISO-8859-1')

#### 💡**Tip**
- Use encoding='ISO-8859-1' only if you face errors with utf-8.<br>
os.path.join...
- This ensures your notebook stays portable, and works across machines or collaborators using the same project layout.

In [4]:
# Display the first few rows of datasets
{
    "Customers" : customers.head(), # type: ignore
    "Products" : products.head(), # type: ignore
    "Sales" : sales.head(), # type: ignore
    "Stores" : stores.head(), # type: ignore
    "Exchange_Rates" : exchange_rates.head(), # type: ignore
    "Data_Dictionary" : data_dict.head() # type: ignore
}

{'Customers':    CustomerKey  Gender               Name            City State Code  \
 0          301  Female      Lilly Harding  WANDEARAH EAST         SA   
 1          325  Female       Madison Hull      MOUNT BUDD         WA   
 2          554  Female      Claire Ferres       WINJALLOK        VIC   
 3          786    Male  Jai Poltpalingada    MIDDLE RIVER         SA   
 4         1042    Male    Aidan Pankhurst   TAWONGA SOUTH        VIC   
 
                State Zip Code    Country  Continent    Birthday  
 0    South Australia     5523  Australia  Australia    7/3/1939  
 1  Western Australia     6522  Australia  Australia   9/27/1979  
 2           Victoria     3380  Australia  Australia   5/26/1947  
 3    South Australia     5223  Australia  Australia   9/17/1957  
 4           Victoria     3698  Australia  Australia  11/19/1965  ,
 'Products':    ProductKey                         Product Name    Brand   Color  \
 0           1  Contoso 512MB MP3 Player E51 Silver  Contoso

**Datasets Summary**<br>

1. Customer.csv<br>
    - Clean layout with location and birthdate information<br>

2. Products.csv<br>
    - Costs are in string format ("$6.62")-need conversion to float<br>

3. Sales.csv<br>
    - Has some missing delivery dates<br>
    - Dates are in string format-should be parsed<br>

4. Store.csv<br>
    - Open Date needs datetime coversion<br>

5. Exchange_Rate.csv<br>
    - Dates in string format-should be parsed<br>

In [6]:
import os

# Define raw data path (adjust based on your local structure)
raw_data_path = "../data/raw"

# Load datasets
customers = pd.read_csv(os.path.join(raw_data_path, "Customers.csv"), encoding='ISO-8859-1')
products = pd.read_csv(os.path.join(raw_data_path, "Products.csv"), encoding='ISO-8859-1')
sales = pd.read_csv(os.path.join(raw_data_path, "Sales.csv"), encoding='ISO-8859-1')
stores = pd.read_csv(os.path.join(raw_data_path, "Stores.csv"), encoding='ISO-8859-1')
exchange_rates = pd.read_csv(os.path.join(raw_data_path, "Exchange_Rates.csv"), encoding='ISO-8859-1')

# Convert date columns to datetime
customers['Birthday'] = pd.to_datetime(customers['Birthday'], errors='coerce')
sales ['Order Date'] = pd.to_datetime(sales['Order Date'], errors='coerce')
sales ['Delivery Date'] = pd.to_datetime(sales['Delivery Date'], errors='coerce')
stores ['Open Date'] = pd.to_datetime(stores['Open Date'], errors='coerce')
exchange_rates ['Date'] = pd.to_datetime(exchange_rates['Date'], errors='coerce')

# Convert currency columns from string to float
products['Unit Cost USD'] = products['Unit Cost USD'].replace('[\$,]', '', regex=True).astype(float) # type: ignore
products['Unit Price USD'] = products['Unit Price USD'].replace('[\$,]', '', regex=True).astype(float) # type: ignore

# Confirm column type changes
column_types = {
    "Cutsomers": customers.dtypes,
    "Products" : products.dtypes,
    "Sales" : sales.dtypes,
    "Stores" : stores.dtypes,
    "Exchange_Rates" : exchange_rates.dtypes
}

# Check conversion
print(customers['Birthday'].dtypes)
print(products[['Unit Cost USD', 'Unit Price USD']].dtypes)
print(sales[['Order Date', 'Delivery Date']].dtypes)
print(stores['Open Date'].dtypes)
print(exchange_rates['Date'].dtypes)

datetime64[ns]
Unit Cost USD     float64
Unit Price USD    float64
dtype: object
Order Date       datetime64[ns]
Delivery Date    datetime64[ns]
dtype: object
datetime64[ns]
datetime64[ns]


<>:21: SyntaxWarning: invalid escape sequence '\$'
<>:22: SyntaxWarning: invalid escape sequence '\$'
<>:21: SyntaxWarning: invalid escape sequence '\$'
<>:22: SyntaxWarning: invalid escape sequence '\$'
C:\Users\VivekKumarDuggal\AppData\Local\Temp\ipykernel_13028\957043412.py:21: SyntaxWarning: invalid escape sequence '\$'
  products['Unit Cost USD'] = products['Unit Cost USD'].replace('[\$,]', '', regex=True).astype(float) # type: ignore
C:\Users\VivekKumarDuggal\AppData\Local\Temp\ipykernel_13028\957043412.py:22: SyntaxWarning: invalid escape sequence '\$'
  products['Unit Price USD'] = products['Unit Price USD'].replace('[\$,]', '', regex=True).astype(float) # type: ignore


🔍 **Check missing values in Each Dataset**

In [7]:
# Check for missing values
print("Customers:\n", customers.isnull().sum(), "\n")
print("Products:\n", products.isnull().sum(), "\n")
print("Sales:\n", sales.isnull().sum(), "\n")
print("Stores:\n", stores.isnull().sum(), "\n")
print("Exchange Rates:\n", exchange_rates.isnull().sum(), "\n")

Customers:
 CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64 

Products:
 ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64 

Sales:
 Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64 

Stores:
 StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64 

Exchange Rates:
 Date        0
Currency    0
Exchange    0
dtype: int64 



✅ **Missing Values Handling Plan**<br>

Customers<br>
- Missing State Code: 10 rows
- Not Critical for analysis (state name is present)
- Fill with "UNKNOWN" for completeness

Products<br>
- No missing value - good to go!

Sales<br>
- Missing Delivery Date: 49,719 rows
- This indicate pending or incomplete orders.
- Carete a flag for analysis (e.g. DeliveryMissing)

Stores<br>
- Missing Square Meter: 1 row
- Fill with mean value

Exchange Rates<br>
- No missing value - good to go!

In [8]:
# Customers: Fill missing state codes
customers['State Code'].fillna('UNKNOWN', inplace=True)

# Sales: Flag missing delivery dates
sales['DeliveryMissing'] = sales['Delivery Date'].isnull()

# Stores: Fill missing square meter with average
stores['Square Meters'].fillna(stores['Square Meters'].mean(), inplace=True)

C:\Users\VivekKumarDuggal\AppData\Local\Temp\ipykernel_13028\232980834.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stores['Square Meters'].fillna(stores['Square Meters'].mean(), inplace=True)


### 🔗**Datasets Merging Plan**
- Build a **master sales dataset** by joining related tabels step-by-step 
    - Start with sales **(central tabel)**
    - Merge with customers using **customer key**
    - Merge with products using **product key**
    - Merge with stores using **store key**
    - Merge with exchange rates on: 
        - Order Date - Date
        - Currency Code - Currency

In [9]:
# Step-by-step merging
sales_full = sales.merge(customers, on='CustomerKey', how='left')
sales_full = sales_full.merge(products, on='ProductKey', how='left')
sales_full = sales_full.merge(stores, on='StoreKey', how='left')
sales_full = sales_full.merge(
    exchange_rates,
    left_on=['Order Date', 'Currency Code'],
    right_on=['Date', 'Currency'],
    how='left'
)
sales_full.drop(columns=['Date', 'Currency'], inplace=True)

Matches **sales** with the **correct exchange rate** using:<br>
- Order Date from Sales - Date in exchange_rates<br>
- Currency Code - Currency<br>

This enables calculating sales values in a **base currency** (like USD).

After merging, you now have:<br>
- Order Date, Currency Code from sales<br>
- Date, Currency from exchange rates (duplicate info)<br>
so you remove the Date and Currency columns from the exchange table.<br>

**✅ Result: sales_full**


In [11]:
# show a sample of the merged dataset
sales_full_sample = sales_full.sample(5, random_state=42) # type: ignore
print(sales_full_sample)

# show data types and missing values summary
missing_summary = sales_full.isnull().sum()
data_types_summary = sales_full.dtypes
print("\nMissing values:\n", sales_full.isnull().sum())
print("\nData types:\n", sales_full.dtypes)

       Order Number  Line Item Order Date Delivery Date  CustomerKey  \
59457       2050004          2 2020-08-11    2020-08-13      1868176   
10102        913006          1 2017-07-01           NaT      1948036   
31319       1494037          6 2019-02-02           NaT      1816342   
7822         770007          1 2017-02-08           NaT       642546   
42697       1717014          1 2019-09-13           NaT      1692115   

       StoreKey  ProductKey  Quantity Currency Code  DeliveryMissing  ...  \
59457         0        1275         1           USD            False  ...   
10102        59         105         1           USD             True  ...   
31319        49         448         4           USD             True  ...   
7822         18        1582         3           EUR             True  ...   
42697        50        1595         8           USD             True  ...   

      Unit Price USD SubcategoryKey                       Subcategory  \
59457          52.13           

**Inspect This Flag:**

In [12]:
sales_full[sales_full['DeliveryMissing']].sample(5, random_state=1)

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code,DeliveryMissing,...,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category,Country_y,State_y,Square Meters,Open Date,Exchange
14054,1084021,6,2017-12-19,NaT,1256108,48,1141,2,USD,True,...,436.20,402,Digital SLR Cameras,4,Cameras and camcorders,United States,Idaho,1540.0,2012-12-15,1.0000
46549,1778020,1,2019-11-13,NaT,1041126,42,1156,1,GBP,True,...,1600.00,405,Camcorders,4,Cameras and camcorders,United Kingdom,North Down,1900.0,2009-12-15,0.7792
45236,1757033,3,2019-10-23,NaT,1411283,65,209,1,USD,True,...,699.00,203,Home Theater System,2,TV and Video,United States,West Virginia,1785.0,2012-01-01,1.0000
21424,1319011,2,2018-08-11,NaT,1758421,57,1503,2,USD,True,...,269.00,504,Smart phones & PDAs,5,Cell phones,United States,New Mexico,1645.0,2010-06-03,1.0000
7349,748002,1,2017-01-17,NaT,1233674,61,1632,3,USD,True,...,17.99,602,Movie DVD,6,"Music, Movies and Audio Books",United States,South Carolina,2000.0,2012-12-15,1.0000


In [13]:
# Save full merged dataset
clean_data_path = "../data/clean"
sales_full.to_csv(os.path.join(clean_data_path, "sales_full.csv"), index=False)

In [16]:
# 1. Filter Completed Deliveries
delivered = sales_full[~sales_full['DeliveryMissing']]
delivered.to_csv(os.path.join(clean_data_path, "sales_delivered.csv"), index=False)

# 2. Filter undeliveries
undelivered = sales_full[sales_full['DeliveryMissing']]
undelivered.to_csv(os.path.join(clean_data_path, "sales_undelivered.csv"), index=True)

completion_rate = 1 - sales_full['DeliveryMissing'].mean()
print(f"Delivery completion rate: {completion_rate:.2%}")

Delivery completion rate: 20.94%
